In [1]:
import pandas as pd
import os
from glob import glob

# Get average on regions un signed

In [27]:

def get_avg_on_reg( csv_file   ):
    df = pd.read_csv(csv_file)

    for i in range(3):
        region_code=str(i+1)
        # Filter rows where 'name' ends with '3'
        filtered = df[df['Animal Name'].astype(str).str.endswith(region_code)]

        # Compute average error
        avg_error = filtered['Abs Error rate enh'].mean()

        print(f"Average error for rows ending with {region_code}: {avg_error}")


In [28]:
def get_avg_on_reg(csv_file):
    df = pd.read_csv(csv_file)
    results = []
    all_filtered = pd.DataFrame()  # For computing total average

    for i in range(3):
        region_code = str(i + 1)
        filtered = df[df['Animal Name'].astype(str).str.endswith(region_code)]
        avg_error = filtered['Abs Error rate enh'].mean()
        results.append(avg_error)
        all_filtered = pd.concat([all_filtered, filtered])

    # Compute total average over all regions
    avg_total = all_filtered['Abs Error rate enh'].mean()
    results.append(avg_total)

    return results

In [33]:

# Folder containing your CSV files
folder_path = "./results/Networks_AAE_ASE_comparisons/"
save_csv_address='./results/'

csv_files = glob(os.path.join(folder_path, "*.csv"))

# Store results
all_results = []

for file in csv_files:
    avg_errors = get_avg_on_reg(file)
    all_results.append([os.path.basename(file)] + avg_errors)

# Convert to DataFrame
results_df = pd.DataFrame(all_results, columns=["filename", "avg_region_1", "avg_region_2", "avg_region_3", "avg_total"])

# Save to CSV
results_df.to_csv(os.path.join(save_csv_address, "average_errors_per_region_train_test_signed.csv"), index=False)

print("Done! Results saved to 'average_errors_per_region.csv'")


Done! Results saved to 'average_errors_per_region.csv'


# Get regions signed

In [ ]:
import pandas as pd
import os
import glob

def get_avg_on_reg(csv_file):
    df = pd.read_csv(csv_file)
    results = []

    # Create a combined DataFrame for total averages
    all_filtered = pd.DataFrame()

    # Loop over regions 1–3
    for i in range(3):
        region_code = str(i + 1)
        filtered = df[df['Animal Name'].astype(str).str.endswith(region_code)]

        # Mean absolute error (unsigned)
        avg_abs_error = filtered['Abs Error rate enh'].mean()

        # Mean signed error (can be negative or positive)
        avg_signed_error = filtered['Error rate enh'].mean()

        # Mean overestimation (positive signed errors)
        avg_over = filtered.loc[filtered['Error rate enh'] > 0, 'Error rate enh'].mean()

        # Mean underestimation (negative signed errors)
        avg_under = filtered.loc[filtered['Error rate enh'] < 0, 'Error rate enh'].mean()

        results.append([
            avg_abs_error, avg_signed_error, avg_over, avg_under
        ])
        all_filtered = pd.concat([all_filtered, filtered])

    # Compute overall (total) averages
    total_abs = all_filtered['Abs Error rate enh'].mean()
    total_signed = all_filtered['Error rate enh'].mean()
    total_over = all_filtered.loc[all_filtered['Error rate enh'] > 0, 'Error rate enh'].mean()
    total_under = all_filtered.loc[all_filtered['Error rate enh'] < 0, 'Error rate enh'].mean()

    results.append([total_abs, total_signed, total_over, total_under])

    return results


In [34]:

# Example usage
folder_path = "./results/Networks_AAE_ASE_comparisons/"
save_csv_address='./results/'
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

all_results = []

for file in csv_files:
    avg_errors = get_avg_on_reg(file)
    # Flatten results for 3 regions + total (4 rows × 4 cols)
    flat = [os.path.basename(file)]
    for row in avg_errors:
        flat.extend(row)
    all_results.append(flat)

# Create DataFrame
columns = ["filename"] + \
           [f"region_{i}_{col}" for i in range(1, 4) for col in ["abs", "signed", "over", "under"]] + \
           [f"total_{col}" for col in ["abs", "signed", "over", "under"]]

results_df = pd.DataFrame(all_results, columns=columns)

# Save to CSV
results_df.to_csv(os.path.join(save_csv_address, "average_signed_abs_over_under_errors.csv"), index=False)

print("✅ Results saved to 'average_signed_abs_over_under_errors.csv'")


✅ Results saved to 'average_signed_abs_over_under_errors.csv'
